In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
import lime
import lime.lime_tabular
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
dataset_folder = 'Datasets/drugs/'
train = pd.read_csv(str(dataset_folder+'train.csv'))
test = pd.read_csv(str(dataset_folder+'test.csv'))

In [32]:
df_all = pd.concat([train,test]).reset_index()
del df_all['index']

In [35]:
df_all.replace('', np.nan, inplace=True)
print(df_all.shape)
df_all.dropna(inplace=True)
print(df_all.shape)

(4143, 2)
(4141, 2)


In [33]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.model_selection import train_test_split

In [34]:
stops = set(stopwords.words('english'))
not_stop = ["aren't","couldn't","didn't","doesn't","don't","hadn't","hasn't","haven't","isn't","mightn't","mustn't","needn't","no","nor","not","shan't","shouldn't","wasn't","weren't","wouldn't"]
for i in not_stop:
    if i in stops:
        stops.remove(i)
    
stemmer = SnowballStemmer('english')
def review_to_words(raw_review):
    review_text = raw_review
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    words = letters_only.lower().split()
    meaningful_words = [w for w in words if not w in stops]
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    return( ' '.join(stemming_words))


In [36]:
%time df_all['sideEffectsReview'] = df_all['sideEffectsReview'].apply(review_to_words)

Wall time: 1.49 s


In [40]:
df_all.sideEffects.unique()

array(['Mild Side Effects', 'Severe Side Effects', 'No Side Effects',
       'Extremely Severe Side Effects', 'Moderate Side Effects'],
      dtype=object)

In [41]:
def categorize_rating(x):
    if x in ['Mild Side Effects', 'No Side Effects']:
        return 0
    else:
        return 1

In [42]:
df_all['sideEffects'] = df_all["sideEffects"].apply(categorize_rating)

In [49]:
df_train, df_test = train_test_split(df_all, test_size=0.2, random_state=42) 

In [136]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None,
                             preprocessor = None, 
                             stop_words = None, 
                             min_df = 2,
                             ngram_range = (1,1),
                             max_features = 5000
                            )

In [91]:
%time X_train = vectorizer.fit_transform(df_train['sideEffectsReview'])
%time X_test = vectorizer.transform(df_test['sideEffectsReview'])

Wall time: 72.2 ms
Wall time: 26.9 ms


In [92]:
y_train = df_train['sideEffects']
y_test = df_test['sideEffects']

In [138]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=3, random_state=1)
c.fit(X_train, y_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(y_train, predict_fn(X_train)))
print('Test', sklearn.metrics.accuracy_score(y_test, predict_fn(X_test)))

Train 0.9978864734299517
Test 0.7949336550060314


In [139]:
class_names = ['negative','positive']

In [97]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
pip_line = make_pipeline(vectorizer, c)

In [98]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [120]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in range(len(xtest)):
        exp = explainer.explain_instance(xtest.iloc[i], pip_line.predict_proba, num_features=5, top_labels=1)
        exp1.append(np.array(exp.as_map()[exp.available_labels()[0]]))
    return np.array(exp1)
exp_fn_wrap = lambda x: exp_fn_blk(x, exp_fn)

In [140]:
print(df_test.shape)
unique_df = df_test.drop_duplicates()
print(unique_df.shape)

(829, 2)
(751, 2)


In [143]:
%time exp1 = exp_fn_blk(unique_df.sideEffectsReview.iloc[0:400], exp_fn)
print('done exp1')
%time exp2 = exp_fn_blk(unique_df.sideEffectsReview.iloc[0:400], exp_fn)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Wall time: 6min 17s
done exp1


C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Wall time: 6min 9s


In [122]:
i = calc_identity(exp1,exp2)
print(i)

(99.5, 2, 400)


In [123]:
s = calc_separability(exp1)
print(s)

(0, 400, 160000, 0.0)


In [124]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i][j,0])] = exp[i][j,1]
    return enc_exp

In [127]:
enc1 = enc_exp(exp1, vectorizer.max_features)
enc2 = enc_exp(exp2, vectorizer.max_features)

In [142]:
sb = calc_stability(enc1, unique_df.sideEffects.iloc[:400].values)
print(sb)

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


(198, 400)
